# From SQL to pandas challenge 10

In [2]:
# import libraries
import pandas as pd
import numpy as np

# load data
# This code is made to load our data stored on Google Drive
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

# Google Drive file ids
files_id = {
    "titles": "1PLdn50N9GRa53ZbuVWo0l47F_IXdvlEm",
    "sales": "1fzFc9rwYmVIPaGOFmhLVxCi3kg19vNU2", 
}

# Read data from Google Drive
sales = pd.read_csv(gd_path(files_id["sales"]), sep=";")
titles = pd.read_csv(gd_path(files_id["titles"]), sep=";")

## 1. Select everything from the sales table and create a new column called "sales_category" to categorise qty:
   
		qty >= 50 high sales
		20 <= qty < 50 medium sales
		qty < 20 low sales

In [3]:
sales["sales_category"] = np.where(sales['qty'] < 20, 'low sales', 
                          np.where((sales['qty'] >= 20) & (sales['qty'] < 50), 'high sales',
                          np.where(sales['qty'] >= 50, 'medium sales', None)))
sales.sample(5)

,stor_id,ord_num,ord_date,qty,payterms,title_id,sales_category
4,7067,D4482,1994-09-14 00:00:00,10,Net 60,PS2091,low sales
2,7066,A2976,1993-05-24 00:00:00,50,Net 30,PC8888,medium sales
16,7896,X999,1993-02-21 00:00:00,35,ON invoice,BU2075,high sales
8,7131,N914008,1994-09-14 00:00:00,20,Net 30,PS2091,high sales
7,7067,P2121,1992-06-15 00:00:00,20,Net 30,TC7777,high sales


In [4]:
#Alternatively
sales['sales_category'] = pd.cut(sales['qty'], bins=[1, 19, 49, 100], labels=['low sales', 'medium sales', 'high sales'])
sales.sample(5)

,stor_id,ord_num,ord_date,qty,payterms,title_id,sales_category
7,7067,P2121,1992-06-15 00:00:00,20,Net 30,TC7777,medium sales
20,8042,QA879.1,1993-05-22 00:00:00,30,Net 30,PC1035,medium sales
11,7131,P3087a,1993-05-29 00:00:00,25,Net 60,PS2106,medium sales
9,7131,N914014,1994-09-14 00:00:00,25,Net 30,MC3021,medium sales
12,7131,P3087a,1993-05-29 00:00:00,15,Net 60,PS3333,low sales


In [6]:
#Alternatively 2
sales.sample(5)
sales["sales_category"] = sales["qty"].apply(lambda x: "high sales" if x >= 50 else "medium sales" if x >= 20 and x < 50 else "low sales")
sales.sample(5)

,stor_id,ord_num,ord_date,qty,payterms,title_id,sales_category,sales_category_1000
14,7896,QQ2299,1993-10-28 00:00:00,15,Net 60,BU7832,low sales,1015
1,6380,722a,1994-09-13 00:00:00,3,Net 60,PS2091,low sales,1003
11,7131,P3087a,1993-05-29 00:00:00,25,Net 60,PS2106,medium sales,1025
3,7066,QA7442.3,1994-09-13 00:00:00,75,ON invoice,PS2091,high sales,1075
12,7131,P3087a,1993-05-29 00:00:00,15,Net 60,PS3333,low sales,1015


In [8]:
##Alternatively La
sales["sales_category_1000"] = sales["qty"].apply(lambda x: x + 1000)
sales.sample(5)

,stor_id,ord_num,ord_date,qty,payterms,title_id,sales_category,sales_category_1000
1,6380,722a,1994-09-13 00:00:00,3,Net 60,PS2091,low sales,1003
12,7131,P3087a,1993-05-29 00:00:00,15,Net 60,PS3333,low sales,1015
0,6380,6871,1994-09-14 00:00:00,5,Net 60,BU1032,low sales,1005
13,7131,P3087a,1993-05-29 00:00:00,25,Net 60,PS7777,medium sales,1025
19,8042,P723,1993-03-11 00:00:00,25,Net 30,BU1111,medium sales,1025


### Hint:

In SQL the syntax is:

```sql
SELECT *,
CASE
    WHEN qty >= 50 THEN "high sales"
    WHEN qty >= 20 THEN "medium sales"
    ELSE "low sales"
END AS sales_category
FROM sales;
```

## 2. Adding to your answer from the previous question. Find out the total amount of books sold (qty) in each sales category 
    i.e. How many books had high sales, how many had medium sales, and how many had low sales

In [9]:
sales.sample(10)
sales_quant = sales.groupby('sales_category').qty.sum()
sales_quant

sales_category
high sales      125
low sales        83
medium sales    285
Name: qty, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT sum(qty),
CASE
	WHEN qty>=50 THEN 'high sales'
    WHEN (qty>=20 AND qty<50) THEN 'medium sales'
    ELSE 'low sales'
END AS sales_category
FROM sales
GROUP BY sales_category;
```

## 3. Adding to your answer from the previous questions: output only those sales categories that have a SUM(qty) greater than 100, and order them in descending order

In [10]:
sales_quant.loc[sales_quant > 100].sort_values(ascending=False)

sales_category
medium sales    285
high sales      125
Name: qty, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT sum(qty),
CASE
    WHEN qty>=50 THEN 'high sales'
    WHEN (qty>=20 AND qty<50) THEN 'medium sales'
    ELSE 'low sales'
END AS sales_category
FROM sales
GROUP BY sales_category
HAVING sum(qty)>100
ORDER BY sum(qty) DESC;
```

## 4. Find out the average book price, per publisher, for the following book types and price categories:
		book types: business, traditional cook and psychology
		price categories: <= 10 super low, <= 15 medium, > 15 high
        
        - When displaying the average prices, use ROUND() to hide decimals.

In [ ]:
titles.columns

Index(['title_id', 'title', 'type', 'pub_id', 'price', 'advance', 'royalty',
       'ytd_sales', 'notes', 'pubdate'],
      dtype='object')

In [ ]:
titles.sample(5)
titles.type.unique()

array(['business', 'mod_cook', 'UNDECIDED', 'popular_comp', 'psychology',
       'trad_cook'], dtype=object)

In [11]:
titles_compackt = round(titles[titles.type.isin(['business','trad_cook','psychology'])][['pub_id',"type","price"]].groupby(["pub_id","type"]).price.mean())
titles_compackt = titles_compackt.to_frame()
titles_compackt["price_ctg"] = titles_compackt.price.apply(lambda x: "hight" if x > 15 else "medium" if x <= 15 and x >= 10 else "super low" )
titles_compackt

price  price_ctg
pub_id type                        
736    business      3.0  super low
       psychology   11.0     medium
877    psychology   22.0      hight
       trad_cook    16.0      hight
1389   business     17.0      hight

In [ ]:
titles[titles.type.isin(['business','trad_cook','psychology'])].groupby("type").price.mean().apply(lambda x: "hight" if x > 15 else "medium" if x <= 15 and x >= 10 else "super low" )

type
business      medium
psychology    medium
trad_cook      hight
Name: price, dtype: object

In [ ]:
titles_compackt["price_ctg"] = titles[titles.type.isin(['business','trad_cook','psychology'])].groupby("type").price.mean().apply(lambda x: "hight" if x > 15 else "medium" if x <= 15 and x >= 10 else "super low" )
titles_compackt

price price_ctg
pub_id type                       
736    business      3.0       NaN
       psychology   11.0       NaN
877    psychology   22.0       NaN
       trad_cook    16.0       NaN
1389   business     17.0       NaN

In [ ]:
# price categories: <= 5 super low, <= 10 low, <= 15 medium, > 15 high


In [ ]:
titles_test = round(titles[titles.type.isin(['business','trad_cook','psychology'])][['pub_id',"type","price"]].groupby(["pub_id","type"]).price.mean())
titles_test = titles_test.to_frame()
titles_test["price_ctg"] = np.where(titles_test['price'] <= 5  ,"super low",
                           np.where(titles_test['price'] <= 10 ,"low",
                           np.where(titles_test['price'] <= 15 ,"medium",
                           np.where(titles_test['price'] > 15  ,"hight", ""))))
titles_test


price  price_ctg
pub_id type                        
736    business      3.0  super low
       psychology   11.0     medium
877    psychology   22.0      hight
       trad_cook    16.0      hight
1389   business     17.0      hight

### Hint:

In SQL the syntax is:

```sql
SELECT
    ROUND(AVG(price)),
    type,
    pub_name,
CASE
    WHEN price <= 5 THEN 'super low'
    WHEN (price > 5 AND price <= 10) THEN 'low'
    WHEN (price > 10 AND price <= 15) THEN 'medium'
    ELSE 'high'
END AS price_category
FROM titles
LEFT JOIN publishers
ON titles.pub_id=publishers.pub_id
GROUP BY
    pub_name,
    type,
    price_category
HAVING
    type IN ('business', 'trad_cook', 'psychology');
```